In [1]:
# Object Classification

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import os
from super_gradients.training import models
from super_gradients.training.datasets.detection_datasets import YoloDarknetFormatDetectionDataset
from super_gradients.training.transforms import DetectionTransform
from super_gradients.training.metrics import DetectionMetrics
'''from super_gradients.training.utils.detection_utils import DetectionCollateFN'''

# -------------------
# 🔧 CONFIGURATION
# -------------------
DATA_YAML = "D:/data/data.yaml"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
N_WAY = 3
K_SHOT = 5
Q_QUERY = 10
INNER_STEPS = 5
INNER_LR = 1e-3
META_LR = 1e-4
EPOCHS = 10

# -------------------
# 🔧 Load YOLO-NAS Backbone
# -------------------
backbone = models.get('yolo_nas_s', pretrained_weights="coco").to(DEVICE)
backbone.head = nn.Identity()  # Remove detection head

class MAMLModel(nn.Module):
    def __init__(self, base_model, head_output=9):  # 9 classes in your dataset
        super().__init__()
        self.backbone = base_model
        self.head = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1000, 512),
            nn.ReLU(),
            nn.Linear(512, head_output)  # logits
        )

    def forward(self, x):
        features = self.backbone(x)
        return self.head(features)

# -------------------
# 📦 Custom Few-Shot Dataset Loader
# -------------------
def load_yolonas_dataset(split):
    transform = DetectionTransform(image_size=416)
    return YoloDarknetFormatDetectionDataset(
        data_dir="D:/data",
        images_dir=os.path.join("images", split),
        labels_dir=os.path.join("labels", split),
        class_names=["airplane", "ship", "storage-tank", "ground-track-field", "harbor", "bridge", "large-vehicle", "small-vehicle", "helicopter"],
        transform=transform
    )

train_dataset = load_yolonas_dataset("train")
val_dataset = load_yolonas_dataset("val")

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=DetectionCollateFN(), num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, collate_fn=DetectionCollateFN(), num_workers=2)

# -------------------
# ⚙️ MAML Meta-Training Loop
# -------------------
meta_model = MAMLModel(backbone).to(DEVICE)
meta_optimizer = optim.Adam(meta_model.parameters(), lr=META_LR)
loss_fn = nn.CrossEntropyLoss()

def clone_model(model):
    import copy
    return copy.deepcopy(model)

for epoch in range(EPOCHS):
    print(f"Epoch {epoch+1}/{EPOCHS}")
    meta_optimizer.zero_grad()
    
    meta_loss = 0.0
    for task_idx in range(5):  # 5 tasks per meta-batch
        learner = clone_model(meta_model)
        learner.train()
        support_set, query_set = [], []

        for class_id in range(N_WAY):
            class_samples = [sample for sample in train_dataset if sample['class'] == class_id]
            support_set += class_samples[:K_SHOT]
            query_set += class_samples[K_SHOT:K_SHOT + Q_QUERY]

        # Inner-loop: Fine-tune learner
        inner_optimizer = optim.SGD(learner.parameters(), lr=INNER_LR)
        for _ in range(INNER_STEPS):
            for sample in support_set:
                img = sample['image'].unsqueeze(0).to(DEVICE)
                label = torch.tensor([sample['class']]).to(DEVICE)
                output = learner(img)
                loss = loss_fn(output, label)
                inner_optimizer.zero_grad()
                loss.backward()
                inner_optimizer.step()

        # Outer-loop: Query loss
        for sample in query_set:
            img = sample['image'].unsqueeze(0).to(DEVICE)
            label = torch.tensor([sample['class']]).to(DEVICE)
            output = learner(img)
            meta_loss += loss_fn(output, label)

    # Meta-optimizer step
    meta_loss /= (N_WAY * Q_QUERY * 5)
    meta_loss.backward()
    meta_optimizer.step()
    print(f"Meta Loss: {meta_loss.item():.4f}")

# -------------------
# ✅ Validation / Test
# -------------------
meta_model.eval()
correct = total = 0

with torch.no_grad():
    for batch in val_loader:
        images = batch['images'].to(DEVICE)
        labels = batch['labels']
        outputs = meta_model(images)
        preds = outputs.argmax(dim=1).cpu()
        correct += (preds == labels).sum().item()
        total += len(labels)

print(f"Validation Accuracy: {100 * correct / total:.2f}%")


URLError: <urlopen error [Errno 11001] getaddrinfo failed>

In [4]:
# Object Detection 

import torch
import torch.nn as nn
import torch.optim as optim
import os
import random
from torch.utils.data import DataLoader
from super_gradients.training import models
from super_gradients.training.datasets.detection_datasets import YoloDarknetFormatDetectionDataset
from super_gradients.training.utils.detection_utils import DetectionCollateFN
from super_gradients.training.losses import YoloNASLoss
from copy import deepcopy

# -------------------
# 🔧 CONFIG
# -------------------
DATA_DIR = "D:/data"
N_WAY = 3
K_SHOT = 5
Q_QUERY = 5
INNER_LR = 1e-3
META_LR = 1e-4
INNER_STEPS = 5
EPOCHS = 10
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# -------------------
# 📦 Dataset Loader
# -------------------
def load_dataset(split):
    return YoloDarknetFormatDetectionDataset(
        data_dir=DATA_DIR,
        images_dir=os.path.join("images", split),
        labels_dir=os.path.join("labels", split),
        class_names=[
            "airplane", "ship", "storage-tank", "ground-track-field", "harbor",
            "bridge", "large-vehicle", "small-vehicle", "helicopter"
        ],
    )

train_dataset = load_dataset("train")
val_dataset = load_dataset("val")

# -------------------
# 🧠 MAML Detector
# -------------------
class MAMLDetector(nn.Module):
    def __init__(self, num_classes=9):
        super().__init__()
        self.model = models.get('yolo_nas_s', pretrained_weights="coco")
        self.loss_fn = YoloNASLoss()
        self.model.to(DEVICE)
    
    def forward(self, x):
        return self.model(x)

    def compute_loss(self, preds, targets):
        return self.loss_fn(preds, targets)

    def clone(self):
        return deepcopy(self)

# -------------------
# 🔁 Meta-Training Loop
# -------------------
meta_model = MAMLDetector().to(DEVICE)
meta_optimizer = optim.Adam(meta_model.parameters(), lr=META_LR)

def sample_task(dataset, n_way, k_shot, q_query):
    class_to_samples = {}
    for i, sample in enumerate(dataset):
        for obj in sample['target']:
            class_id = int(obj[-1])
            class_to_samples.setdefault(class_id, []).append(sample)
    
    chosen_classes = random.sample(list(class_to_samples.keys()), n_way)
    support, query = [], []
    for c in chosen_classes:
        samples = random.sample(class_to_samples[c], k_shot + q_query)
        support.extend(samples[:k_shot])
        query.extend(samples[k_shot:])
    return support, query

def prepare_batch(samples):
    images = torch.stack([s['image'] for s in samples]).to(DEVICE)
    targets = [s['target'].to(DEVICE) for s in samples]
    return images, targets

for epoch in range(EPOCHS):
    print(f"\n[Epoch {epoch+1}]")
    meta_optimizer.zero_grad()
    meta_loss = 0.0

    for task_id in range(5):  # 5 tasks per meta-batch
        learner = meta_model.clone()
        inner_optimizer = optim.SGD(learner.parameters(), lr=INNER_LR)
        
        support_set, query_set = sample_task(train_dataset, N_WAY, K_SHOT, Q_QUERY)

        # -----------------------
        # 🔁 Inner Loop Training
        # -----------------------
        learner.train()
        for _ in range(INNER_STEPS):
            s_imgs, s_targets = prepare_batch(support_set)
            s_preds = learner(s_imgs)
            loss = learner.compute_loss(s_preds, s_targets)
            inner_optimizer.zero_grad()
            loss.backward()
            inner_optimizer.step()

        # -----------------------
        # 📤 Query Set Evaluation
        # -----------------------
        q_imgs, q_targets = prepare_batch(query_set)
        q_preds = learner(q_imgs)
        loss_q = learner.compute_loss(q_preds, q_targets)
        meta_loss += loss_q

    meta_loss /= 5
    meta_loss.backward()
    meta_optimizer.step()
    print(f"Meta Loss: {meta_loss.item():.4f}")

# -------------------
# ✅ Evaluation
# -------------------
from super_gradients.training.metrics.detection_metrics import DetectionMetrics

meta_model.eval()
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, collate_fn=DetectionCollateFN())

evaluator = DetectionMetrics(num_classes=9)
with torch.no_grad():
    for batch in val_loader:
        images = batch['images'].to(DEVICE)
        targets = batch['targets']
        preds = meta_model(images)
        evaluator.update(preds, targets)

map50 = evaluator.compute()['mAP_50']
print(f"\n📈 mAP@50 on Validation Set: {map50:.4f}")


ImportError: cannot import name 'DetectionCollateFN' from 'super_gradients.training.utils.detection_utils' (C:\Users\debsp\AppData\Local\Programs\Python\Python310\lib\site-packages\super_gradients\training\utils\detection_utils.py)

In [2]:
# MAML FSOD+Classification Code (YOLO-NAS)

import torch
import torch.nn as nn
import torch.optim as optim
import os
import random
from torch.utils.data import DataLoader
from copy import deepcopy
from super_gradients.training import models
from super_gradients.training.datasets.detection_datasets import YoloDarknetFormatDetectionDataset
from super_gradients.training.losses import YoloNASLoss
from super_gradients.training.utils.detection_utils import DetectionCollateFN
from super_gradients.training.metrics.detection_metrics import DetectionMetrics

# -------------------
# ⚙️ Config
# -------------------
DATA_DIR = "D:/data"
NUM_CLASSES = 9
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
N_WAY = 3
K_SHOT = 5
Q_QUERY = 5
INNER_STEPS = 3
INNER_LR = 1e-3
META_LR = 1e-4
EPOCHS = 10

# -------------------
# 📦 Dataset Loader
# -------------------
def load_dataset(split):
    return YoloDarknetFormatDetectionDataset(
        data_dir=DATA_DIR,
        images_dir=os.path.join("images", split),
        labels_dir=os.path.join("labels", split),
        class_names=[
            "airplane", "ship", "storage-tank", "ground-track-field", "harbor",
            "bridge", "large-vehicle", "small-vehicle", "helicopter"
        ],
    )

train_dataset = load_dataset("train")
val_dataset = load_dataset("val")

# -------------------
# 🧠 MAML-Compatible Detector
# -------------------
class MAMLDetector(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = models.get("yolo_nas_s", pretrained_weights="coco")
        self.loss_fn = YoloNASLoss()
        self.model.to(DEVICE)

    def forward(self, x):
        return self.model(x)

    def compute_loss(self, preds, targets):
        return self.loss_fn(preds, targets)

    def clone(self):
        return deepcopy(self)

# -------------------
# 🎯 Few-Shot Task Sampling
# -------------------
def sample_task(dataset, n_way, k_shot, q_query):
    class_to_samples = {}
    for s in dataset:
        for ann in s['target']:
            class_id = int(ann[-1])
            class_to_samples.setdefault(class_id, []).append(s)

    chosen_classes = random.sample(list(class_to_samples.keys()), n_way)
    support, query = [], []

    for c in chosen_classes:
        examples = random.sample(class_to_samples[c], k_shot + q_query)
        support.extend(examples[:k_shot])
        query.extend(examples[k_shot:])

    return support, query

def prepare_batch(samples):
    images = torch.stack([s['image'] for s in samples]).to(DEVICE)
    targets = [s['target'].to(DEVICE) for s in samples]
    return images, targets

# -------------------
# 🔁 Meta-Learning Loop
# -------------------
meta_model = MAMLDetector().to(DEVICE)
meta_optimizer = optim.Adam(meta_model.parameters(), lr=META_LR)

for epoch in range(EPOCHS):
    print(f"\n[Epoch {epoch+1}]")
    meta_optimizer.zero_grad()
    meta_loss = 0.0

    for task in range(5):  # 5 tasks per meta-batch
        learner = meta_model.clone()
        inner_optimizer = optim.SGD(learner.parameters(), lr=INNER_LR)

        support_set, query_set = sample_task(train_dataset, N_WAY, K_SHOT, Q_QUERY)

        # ----------- Inner Loop -----------
        learner.train()
        for _ in range(INNER_STEPS):
            s_imgs, s_targets = prepare_batch(support_set)
            s_preds = learner(s_imgs)
            s_loss = learner.compute_loss(s_preds, s_targets)
            inner_optimizer.zero_grad()
            s_loss.backward()
            inner_optimizer.step()

        # ----------- Outer Loop -----------
        q_imgs, q_targets = prepare_batch(query_set)
        q_preds = learner(q_imgs)
        q_loss = learner.compute_loss(q_preds, q_targets)
        meta_loss += q_loss

    meta_loss /= 5
    meta_loss.backward()
    meta_optimizer.step()
    print(f"Meta Loss: {meta_loss.item():.4f}")

# -------------------
# ✅ Evaluation: Detection + Classification
# -------------------
meta_model.eval()
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, collate_fn=DetectionCollateFN())
evaluator = DetectionMetrics(num_classes=NUM_CLASSES)

with torch.no_grad():
    for batch in val_loader:
        images = batch['images'].to(DEVICE)
        targets = batch['targets']
        preds = meta_model(images)
        evaluator.update(preds, targets)

results = evaluator.compute()
print(f"\n📊 Validation Results:")
print(f"mAP@50: {results['mAP_50']:.4f}")
print(f"mAP@50-95: {results['mAP_50_95']:.4f}")


ImportError: cannot import name 'YoloNASLoss' from 'super_gradients.training.losses' (C:\Users\debsp\AppData\Local\Programs\Python\Python310\lib\site-packages\super_gradients\training\losses\__init__.py)

In [ ]:
try:
    import super_gradients
    print("✅ super_gradients is installed.")
except ImportError:
    print("❌ super_gradients is NOT installed.")


In [ ]:
import urllib.request

try:
    urllib.request.urlopen("https://www.google.com", timeout=5)
    print("Internet access: OK")
except Exception as e:
    print(f"Internet access: FAIL — {e}")
print ("kell")